<center><h1>Proyecto: Predicción de precios de venta de casas</h1></center>

## 1. Introducción.
Comenzamos desarrollando la intuición para el aprendizaje basado en modelos, exploramos cómo funcionaba el modelo de regresión lineal, comprendimos cómo funcionaban los dos enfoques diferentes para el ajuste de modelos y algunas técnicas para limpiar, transformar y seleccionar características. 

Trabajaremos con datos de vivienda de la ciudad de Ames, Iowa, Estados Unidos, de 2006 a 2010. Puede leer más sobre por qué se recopilaron los datos <a href="https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627" target="_blank">aquí</a>. También puede leer sobre las diferentes columnas en la descripción.

Comencemos analizando el conjunto de rutinas que nos permitirá iterar rápidamente en diferentes modelos:

<a id="fig:pipeline"></a>
<img src="figs/pipeline.svg" height="150" width="200"  alt="pipeline"/>


### Ejercicio
- Importe pandas, matplotlib y numpy. Importe también las clases que necesita de scikit-learn.
- Lea `AmesHousing.tsv` en un DataFrame de pandas.
- Para las siguientes funciones, recomendamos crearlas en las primeras celdas del notebook. De esta manera, puede agregar celdas al final del notebook para hacer experimentos y actualizar las funciones en estas celdas.
    - Cree una función llamada `transform_features()` que, por ahora, solo devuelve dataframe de `train`.
    - Cree una función llamada `select_features()` que, por ahora, solo devuelve las columnas `Gr Liv Area` y `SalePrice` del dataframe `train`.
    - Cree una función llamada `train_and_test()` que, por ahora:
        - Selecciona las primeras `1460` filas de `data` y las asigna a `train`.
        - Selecciona las filas restantes de `data` y asignar a `test`.
        - Entrena un modelo utilizando todas las columnas numéricas excepto la columna `SalePrice` (la columna target) del dataframe devuelto por `select_features()`
        - Prueba el modelo en el conjunto de prueba y devuelve el valor RMSE.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,170000


In [3]:
def transform_features(df):
    return df

In [4]:
def select_features():
    return df[['Gr Liv Area', 'SalePrice']]

In [5]:
def train_and_test(df,features, target):
    train = df[:1460].copy()
    test = df[1460:].copy()
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    
    predictions = lr.predict(test[features])
    mean_squared_error(predictions, test[target])
    
    return np.sqrt(mean_squared_error)    

## 2. Ingeniería de características.
Ahora comencemos a eliminar características con muchos valores faltantes, profundizando en características categóricas potenciales y transformando texto y columnas numéricas. Actualice `transform_features()` para que se elimine cualquier columna del dataframe con más del 25 % (u otro valor límite) de valores faltantes. También debe eliminar cualquier columna que no aporte información sobre la venta (por ejemplo, como el año en que ocurrió la venta). En general, el objetivo de esta función es:

- eliminar variables que no queremos usar en el modelo, solo en función de la cantidad de valores faltantes o falta de información
- transformar características en el formato adecuado (numérico a categórico, escalar numérico, completar valores faltantes, etc.)
- crear nuevas variables combinando otras variables (características)

Se recomienda familiarizarse más con las columnas restantes leyendo la documentación, determinando qué transformaciones son necesarias (si las hay) y más. El éxito en el modelado predictivo depende en gran medida de la calidad de las variables que tenga el modelo. Las bibliotecas como scikit-learn demuestran que probar y modificar muchos modelos diferentes sea rápido y fácil, pero limpiar, seleccionar y transformar características sigue siendo más un arte que requiere un poco de ingenio humano.

### Ejercicio
- Agregue algunas celdas para explorar y experimentar con diferentes variables (antes de reescribir estas funciones).
- La función `transform_features()` no debería modificar `train` y, en su lugar, devolver nuevo dataframe por completo. De esta manera, podemos seguir usando `train` en las celdas de experimentación.
- ¿Qué columnas contienen menos del 5 % de valores perdidos?
    - Para las columnas numéricas que cumplan con este criterio, completemos los valores faltantes usando el valor más popular para esa columna.
- ¿Qué nuevas variables podemos crear, que capturen mejor la información en algunas de las variables?
    - Un ejemplo de esto sería la característica `years_until_remod` que creamos anteriormente.
- ¿Qué columnas deben eliminarse por otras razones?
    - ¿Qué columnas no son útiles para el aprendizaje automático?
    - ¿Qué columnas no aportan información sobre la venta final?

## 3. Selección de características.
Después de limpiat y transformar muchas de las características del conjunto de datos, es hora de pasar a la selección de características para características numéricas.

### Ejercicio
- Genere un mapa de calor con la matriz de correlación de las características numéricas en el conjunto de datos de entrenamiento.
    - ¿Qué características se correlacionan fuertemente con nuestra columna objetivo, `SalePrice`?
    - Calcule los coeficientes de correlación para las columnas que parecen correlacionarse bien con `SalePrice`. Debido a que contamos con un [esquema de trabajo](#fig:pipeline), es fácil probar diferentes variables (características) y ver qué variables dan como resultado una mejor puntuación en la validación cruzada.

- ¿Qué columnas del dataframe deben convertirse al tipo de datos categóricos? Todas las columnas que se pueden categorizar como variables nominales son candidatas para convertirse en categóricas. Aquí hay algunas otras cosas en las que debe pensar:
    - Si una columna categórica tiene cientos de valores (o categorías) únicos, ¿debería conservarla? Cuando codifique de forma ficticia esta columna, será necesario volver a agregar cientos de columnas al dataframe.
    - ¿Qué columnas categóricas tienen algunos valores únicos, pero más del 95 % de los valores de la columna pertenecen a una categoría específica? Esto sería similar a una característica numérica de baja varianza (sin variabilidad en los datos para que el modelo los capture).

- ¿Qué columnas actualmente son numéricas pero deben codificarse como categóricas (porque los números no tienen ningún significado semántico)?

- ¿En qué forma podemos explorar columnas categóricas que "se correlacionan" bien con `SalePrice`?
    - Lea <a href="https://machinelearningmastery.com/feature-selection-machine-learning-python/" target="_blank">este blog</a> para conocer algunas estrategias.

- Actualice la lógica de la función `select_features()`. Esta función debe aceptar los dataframes modificados de prueba y entrenamiento que se obtuvieron de la función `transform_features()`.

## 4. Entrenar y probar.
Ahora, para la parte final de nuestro [esquema de trabajo](#fig:pipeline), entrenamiento y pruebas. Al iterar sobre diferentes características, es una buena idea usar una validación simple. Agreguemos un parámetro llamado `k` que controla el tipo de validación cruzada que ocurre.

### Ejercicio
- El parámetro `k` opcional debe aceptar valores enteros, con un valor predeterminado de `0`.
- Cuando `k` sea igual a `0`, realice la validación de exlusión (lo que ya implementamos):
    - Seleccione las primeras `1460` filas y asígnelas a `train`.
    - Seleccione las filas restantes y asígnelas a `test`.
    - Entrena sobre `train` y prueba sobre `test`.
    - Calcule y retorne el RMSE.

- Cuando `k` es igual a `1`, realice una validación cruzada simple:
    - Aleatoriza el orden de las filas en el dataframe.
    - Seleccione las primeras `1460` filas y asígnelas a `fold_one`.
    - Seleccione las filas restantes y asígnelas a `fold_two`.
    - Entrena en fold_one y prueba en `fold_two`.
    - Entrena en fold_two y prueba en `fold_one`.
    - Calcule y devuelva el RMSE promedio.

- Cuando `k` es mayor que `0`, implemente la validación cruzada k-fold usando `k` subconjuntos:
    - Realice la validación cruzada de k-fold usando `k` subconjuntos.
    - Calcule el valor promedio de RMSE y devuelva este valor.

## 5. Próximos pasos.
Estos son algunos de los próximos pasos potenciales que puede tomar:

- Continuar la iteración en la ingeniería de características:
    - Investigue otros enfoques para la ingeniería de características en línea en torno a los datos de vivienda.
    - Visite la página de <a href="https://www.kaggle.com/c/house-prices-advanced-regression-techniques/kernels" target="_blank">kernels de Kaggle</a> para este conjunto de datos para ver los enfoques que otros tomaron.
    Mejore su selección de características:
- Investigue formas de hacer mejor la selección de características con columnas categóricas (algo que no cubrimos).